In [1]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.AlexNet import AlexNet1

Using TensorFlow backend.


# Inisialisasi / Persiapan

In [2]:
fold_index = 0
dataset_dir = 'dataset/bin/GH.227.B.5K'
trained_dir = 'trained/GH.227.B.5K AN1.1'

# Memuat Dataset

In [4]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [5]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [6]:
class IterateChannelParams(GaborParams):
    
    def __init__(self, lambd, sigma, gamma):
        self.lambd = lambd
        self.sigma = sigma
        self.gamma = gammba

    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        num_channel = shape[2]
        
        i += 1
        j += 1
        ratio = j / num_channel
        
        lambd =  np.linspace(self.lambd[0], self.lambd[1], num_channel)[j - 1]
        theta = degree * np.pi / 180
        psi = j
        sigma = np.linspace(self.sigma[0], self.sigma[1], num_channel)[j - 1]
        gamma = np.linspace(self.gamma[0], self.gamma[1], num_channel)[j - 1]
        
        return lambd, theta, psi, sigma, gamma
    
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel Gabor

In [7]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())

# Mendefinisikan Model

In [10]:
ars = AlexNet1(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5
}, loss = 'categorical_crossentropy', optimizer = 'adadelta')
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

model = ars.model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 55, 55, 96)        34944     
_________________________________________________________________
mp1 (MaxPooling2D)           (None, 27, 27, 96)        0         
_________________________________________________________________
c2 (Conv2D)                  (None, 23, 23, 256)       614656    
_________________________________________________________________
mp2 (MaxPooling2D)           (None, 11, 11, 256)       0         
_________________________________________________________________
c3 (Conv2D)                  (None, 11, 11, 384)       885120    
_________________________________________________________________
c4 (Conv2D)                  (None, 11, 11, 384)       1327488   
_________________________________________________________________
c5 (Conv2D)                  (None, 11, 11, 256)       884992    
__________

# Pelatihan

In [ ]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 1, verbose = 1, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 0)